## Dataset Lenght of Stay

Il dataset che abbiamo scelto per il nostro task contiene 100.000 records ognuno dei quali è relativo al ricovero ospedaliero di un paziente.
Gli attributi considerati sono in parte relativi all'identità del paziente, e in parte alla sua condizione clinica e sanitaria. 

È interessante osservare la distribuzione dei diversi attributi.
Un fatto immediatamente evidente è lo sbilanciamento delle etichette di classe: la maggior parte delle durate di ricovero sono inferiori alla metà del valore massimo.

In [98]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator
pd.set_option('future.no_silent_downcasting', True)     # Consente che si possa cambiare il tipo degli attributi nel dataset, servirà in fase di preprocessing


Importando il dataset e visualizzando i primi record ci accorgiamo che ci sono degli attributi data, questi non possono essere elaborati direttamente da Tensorflow ma vanno preprocessati e convertiti in valori numerici.
Assegnamo ad ogni data il corrispondente giorno dell'anno da 1 a 365 (o 366 se l'anno e bisestile)

In [99]:
df = pd.read_csv("LengthOfStay.csv")

dates = df["vdate"]     # Date originali
encoded_dates = [datetime.datetime.strptime(str(date), "%m/%d/%Y").timetuple().tm_yday for date in dates] # Date codificate 
encoding_dict = dict(zip(dates, encoded_dates))     # Creazione dizionario di codifica
df.replace({"vdate": encoding_dict}, inplace=True)  # Sostituzione nella colonna

dates = df["discharged"]
encoded_dates = [datetime.datetime.strptime(str(date), "%m/%d/%Y").timetuple().tm_yday for date in dates]
encoding_dict = dict(zip(dates, encoded_dates))
df.replace({"discharged": encoding_dict}, inplace=True)

df.head()

,eid,vdate,rcount,gender,dialysisrenalendstage,asthma,irondef,pneum,substancedependence,psychologicaldisordermajor,...,glucose,bloodureanitro,creatinine,bmi,pulse,respiration,secondarydiagnosisnonicd9,discharged,facid,lengthofstay
0,1,242,0,F,0,0,0,0,0,0,...,192.476918,12.0,1.390722,30.432418,96,6.5,4,245,B,3
1,2,147,5+,F,0,0,0,0,0,0,...,94.078507,8.0,0.943164,28.460516,61,6.5,1,154,A,7
2,3,266,1,F,0,0,0,0,0,0,...,130.530524,12.0,1.065750,28.843812,64,6.5,2,269,B,3
3,4,222,0,F,0,0,0,0,0,0,...,163.377028,12.0,0.906862,27.959007,76,6.5,1,223,A,1
4,5,355,0,F,0,0,0,1,0,1,...,94.886654,11.5,1.242854,30.258927,67,5.6,2,359,E,4


Sistemate le date, gli altri attributi a cui dovremo dare una codifica numerica sono:
- rcount: trasformeremo il valore 5+ in 5
- gender: trasformeremo M in 0 ed F 1
- facid: trasformeremo le lettere in numeri incrementali


In [100]:
df.replace({"rcount": dict(zip(sorted(df["rcount"].unique()), range(6)))}, inplace=True)   # Codifica del 5+ in 5 e conversione a interi

df.replace({"gender": "M"}, 0, inplace=True)    # Codifica binaria del genere
df.replace({"gender": "F"}, 1, inplace=True)

keys = df["facid"].unique()                        # Estrazione dei valori unici dell'attributo
ints = {key: value for value, key in enumerate(keys)}    # Creazione del dizionario di codifica

df.replace({"facid": ints}, inplace=True)                # Codifica del facid

df.head(10)

,eid,vdate,rcount,gender,dialysisrenalendstage,asthma,irondef,pneum,substancedependence,psychologicaldisordermajor,...,glucose,bloodureanitro,creatinine,bmi,pulse,respiration,secondarydiagnosisnonicd9,discharged,facid,lengthofstay
0,1,242,0,1,0,0,0,0,0,0,...,192.476918,12.0,1.390722,30.432418,96,6.5,4,245,0,3
1,2,147,5,1,0,0,0,0,0,0,...,94.078507,8.0,0.943164,28.460516,61,6.5,1,154,1,7
2,3,266,1,1,0,0,0,0,0,0,...,130.530524,12.0,1.065750,28.843812,64,6.5,2,269,0,3
3,4,222,0,1,0,0,0,0,0,0,...,163.377028,12.0,0.906862,27.959007,76,6.5,1,223,1,1
4,5,355,0,1,0,0,0,1,0,1,...,94.886654,11.5,1.242854,30.258927,67,5.6,2,359,2,4
5,6,332,3,0,0,0,0,0,0,0,...,120.265259,11.0,0.884663,27.875940,83,6.1,1,338,0,6
6,7,271,4,1,0,0,0,0,0,0,...,124.335692,6.0,0.923083,29.928817,68,6.5,4,277,1,6
7,8,156,0,1,0,0,0,0,0,1,...,152.265862,11.0,0.768106,32.999171,63,6.0,3,159,2,3
8,9,187,0,1,0,0,0,0,1,0,...,100.283199,12.0,1.178786,28.802682,69,6.5,1,190,2,3
9,10,245,0,1,0,0,0,0,0,0,...,167.066115,10.0,1.585072,32.503727,65,6.5,0,247,2,2


A questo punto il dataset ha solamente valori numerici e può essere trattato come un tensore.
Sappiamo che le reti neurali necessitano che i tensori da esse elaborati contengano valori tra 0 e 1, dunque è opportuno standardizzare i dati

In [101]:
dataset = tf.
test_ratio = 0.2
val_ratio = 0.2
data_len = len(dataset)

features = dataset[:, 1:-1]
labels = dataset[:, -1]
print(features, labels)

SyntaxError: invalid syntax (753366797.py, line 1)